# First home assignment Social Data Science

### Implement multiple linear regression by gradient descent
Your implementation should be based on numpy arrays. Your implementation can use functions from the numpy library.

Your function should have the signature
linear_regression_gd (X,y,learning_rate) and should return a tuple (mean_squared_error_of_solution, [list_of_optimum_parameters])
Here, X is a numpy array with the values of the explanatory variables, and y is an array with the dependent variable



In [1]:
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

#Display all content in dataframe
pd.set_option('display.max_colwidth', -1)

Formula:

$\beta^T = (b_0, b_1, ...b_j)$

$C(\beta) = \frac{1}{2N}(Y-\beta^T X)^2$

$b_j = b_j - \alpha \frac{1}{N}\sum_i^N(-Y_i + \beta^TX_i)X_{ij}$

In [2]:
# Ref: https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f
"""
    Gradient Descent
    params: 
    -------------
    X: explanatory varialbes
    y: dependent variables
    learning_rate: ratio to update parameter
    max_iter: default 10000, maximum iterations of running updates
    epsilon: default 0.001, threshold to break the loop
    
    returns:
    ---------------
    mse: mean squared error of the sulution
    beta: list of optimum parameters
"""
def linear_regression_gd(X, y, learning_rate, max_iter=10000, epsilon=0.001):
    N = len(y)
    X = np.c_[np.ones(len(X)), X]  # add bias unit
    features = X.shape[1]
    beta = np.array(np.random.random(features)) # randonly choose initial parameters

    for i in range(max_iter):
        pred_y = np.dot(X, beta)
        mse = np.sum(np.square(y - pred_y)) / (2 * N)
        beta = beta - learning_rate * (np.dot(X.T, (pred_y - y))) / N
        new_pred_y = np.dot(X, beta)
        # if the update is so small, finish the gradient descent
        if np.sum(np.square(new_pred_y, pred_y)) < epsilon:
            break

    return (mse, beta.tolist())

### Extend your previous code to implement Stochastic Gradient Descent
Your function should have the signature
linear_regression_sgd (X,y,learning_rate, batch_size)

In [3]:
"""
    Stochastic Gradient Descent
    params: 
    -------------
    X: explanatory varialbes
    y: dependent variables
    learning_rate: ratio to update parameter
    batch_size: number of samples used to train the model
    max_iter: default 10000, maximum iterations of running updates
    epsilon: default 0.001, threshold to break the loop
    
    returns:
    ---------------
    mse: mean squared error of the sulution
    beta: list of optimum parameters
    
"""

def linear_regression_sgd(X, y, learning_rate, batch_size=20, max_iter=10000, epsilon=0.001):
    N = len(y)
    features = X.shape[1] + 1 # one more feature for the bias unit
    instances = X.shape[0]
    beta = np.array(np.random.random(features))
    for i in range(max_iter):
        indices = np.random.choice(range(instances), batch_size)
        X_batch = np.c_[np.ones(len(X[indices])), X[indices]] # add bias unit
        y_batch = y[indices]
        pred_y = np.dot(X_batch, beta)
        mse = np.sum(np.square(y_batch - pred_y)) / (2 * N)
        beta = beta - learning_rate * (np.dot(X_batch.T, (pred_y - y_batch))) / N
        new_pred_y = np.dot(X_batch, beta)
        if np.sum(np.square(new_pred_y, pred_y)) < epsilon:
            break
    return (mse, beta.tolist())

### Load the quality of Government Dataset 
from here: https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan18.csv.
The data is described here
load it into a pandas dataframe and select the following columns:
"cname","wdi_lifexp","wdi_popden","gle_cgdpc","bti_acp", "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion","bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"


In [4]:
url = 'https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan18.csv'
df = pd.read_csv(url)
df = df[["cname","wdi_lifexp","wdi_popden","gle_cgdpc","bti_acp", 
                              "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion",
                              "bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"]]
df.drop(['cname'], axis=1, inplace=True)

In [5]:
df.head()

,wdi_lifexp,wdi_popden,gle_cgdpc,bti_acp,bti_pdi,fh_pair,al_ethnic,al_language,al_religion,bti_aar,vdem_gender,bti_ci,bti_foe,wdi_araland,wdi_forest
0,62.902683,50.176178,1282.6400,3.0,3.0,2,0.769345,0.614146,0.271684,4.0,0.530792,9.0,4.0,11.903011,2.067825
1,77.998390,105.441750,8516.7002,5.0,6.0,9,0.220426,0.039925,0.471852,8.0,0.828077,3.0,7.0,22.467154,28.191971
2,75.635025,16.422152,5402.1699,5.0,3.0,7,0.339400,0.442662,0.009128,5.0,0.770516,7.0,6.0,3.136109,0.818057
3,NaN,168.559570,32367.3300,NaN,NaN,15,0.713946,0.684785,0.232569,NaN,NaN,NaN,NaN,5.957447,34.042553
4,60.806732,21.593380,3771.2000,2.0,3.0,3,0.786720,0.787019,0.627644,3.0,0.680371,4.0,4.0,3.930376,46.507420


### Compute the correlation of all other variables with the life expectancy (wdi_lifexp)

In [6]:
df_corr = df.dropna(axis=0) # drop na values for all rows

df_all = df_corr.drop(['wdi_lifexp'], axis=1)
variables = df_all.columns.tolist() # list of column names

cov = [np.cov(df_corr['wdi_lifexp'], df_all[i])[1, 0] for i in variables]

#calculate correlation coefficiant of wdi_lifexp with other features
pearson_correlation = [np.corrcoef(df_corr['wdi_lifexp'], df_all[i])[1, 0] for i in variables]

# pearson correlation and covariance table
cov_table = pd.DataFrame(cov, columns=['Covariance'], index=variables)
pearson_corr_table = pd.DataFrame(pearson_correlation, columns=['Pearson Correlation'], index=variables)
pd.concat([pearson_corr_table, cov_table],axis=1,join='inner').sort_values('Pearson Correlation', ascending=False)

,Pearson Correlation,Covariance
fh_pair,0.509754,13.432041
gle_cgdpc,0.497971,50262.103316
bti_acp,0.419631,6.291699
vdem_gender,0.200989,0.254124
bti_pdi,0.197603,4.047438
wdi_popden,0.190459,1062.117069
bti_foe,0.153353,2.728130
bti_aar,0.150202,3.071413
wdi_forest,0.087891,14.342895
wdi_araland,0.024516,2.659676


### Apply your own implementations  (GD and SGD)
To model the life expectancy from the population density and GDP per capita.
Compare the results with what you get from scikit learn OR statsmodel libraries.

For this subtask, just remove all countries with missing values in any of these three variables.

In [7]:
df_test = df[['wdi_lifexp', 'wdi_popden', 'gle_cgdpc']].dropna(axis=0)
X = df_test[['wdi_popden', 'gle_cgdpc']].values
y = df_test['wdi_lifexp'].values
# standarize the data
X = preprocessing.scale(X)
y = preprocessing.scale(y)

In [8]:
# use Gradient Decent

params = linear_regression_gd(X, y, 0.01)
my_gd_pred = [sum(r)+params[1][0] for r in np.array(params[1][1:])* X]

print('model intercept: ', params[1][0])
print('model parameters: ', params[1][1:])
print('r2 score: ', r2_score(y, my_gd_pred))


model intercept:  2.8972823673845823e-16
model parameters:  [0.03949066093307813, 0.6245564841445594]
r2 score:  0.4020006760855134


The results of SGD models changes each time when you run it.

In [9]:
# use Stochastic Gradient Descent

params = linear_regression_sgd(X, y, 0.01)
my_sgd_pred = [sum(r)+params[1][0] for r in np.array(params[1][1:])* X]

print('model intercept: ', params[1][0])
print('model parameters: ', params[1][1:])
print('r2 score: ', r2_score(y, my_sgd_pred))

model intercept:  -0.00416427135790511
model parameters:  [0.04797434957849235, 0.610497199183345]
r2 score:  0.4017638490178921


In [10]:
# # Stochastic Gradient Descent from sklearn library
sk_SGD = SGDRegressor(max_iter=10000)
sk_SGD.fit(X, y)
pred_y = sk_SGD.predict(X)

# ### Just compare the r2 score
print('model intercept: ', sk_SGD.intercept_)
print('model parameters: ', sk_SGD.coef_)
print('r2 score: ', r2_score(y, pred_y))

model intercept:  [-7.74421318e-05]
model parameters:  [0.03874848 0.62402598]
r2 score:  0.4019996722677197


### Build regression models to model the life expectancy (wdi_lifexp) in this dataset
from all other mentioned variables.
You can use scikit learn and/or statsmodels libraries for this task.
Standardize variables and fill in missing values appropriately.
Compare linear regression, Ridge regression and Lasso using k-fold-cross validation
Test several parameters for the regularized regressions.


In [11]:
df_reg = df.copy()
df_reg = df_reg.fillna(df_reg.mean())# fill na values with mean value

y = df_reg['wdi_lifexp'].values
X = df_reg.drop(['wdi_lifexp'], axis=1).values
# standarize the data
X = preprocessing.scale(X)
y = preprocessing.scale(y)

In [12]:
# linear regression
fold = [5, 10, 20]
scores = []
lin_reg = LinearRegression()
lin_reg.fit(X, y)
for k in fold: 
    lin_val_score = cross_val_score(lin_reg, X, y, cv=k)
    # calculate accuracy score = mean(validation_score) +/- std(validation_score)
    accuracy = '{0:.3f}+/-{1:.3f}'.format(np.mean(lin_val_score), np.std(lin_val_score))
    # calculate r2 score
    r2_score = '{0:.3f} '.format(lin_reg.score(X, y))
    score = [k, accuracy, r2_score]
    scores.append(score)

pd.DataFrame(np.array(scores), columns=['fold', 'accuracy', 'r2 score'])

,fold,accuracy,r2 score
0,5,0.660+/-0.105,0.699
1,10,0.615+/-0.135,0.699
2,20,0.568+/-0.271,0.699


In [13]:
# Regularization strength
"""
    Model validation to calcuate accuracy score and r2 score
    params:
    -------------
    model: name for model, (Ridge, Lasso)
    X: descriptive features
    y: target features
    fold: default 10, fold for cross validation
    strength: default 0, strength for penalty
    iter: default 1000, number of iterations
    
    returns: 
    --------------
    accuracy: accuracy socre
    model_r2_score: r2 score for the model
"""
def model_validation(model, X, y, fold=10, strength=0, iter=1000):
    
    model_reg = model(alpha=strength, max_iter=iter)
    model_reg.fit(X, y)
    
    model_val_score = cross_val_score(model_reg, X, y, cv=fold)
    model_r2_score = '{0:.3f} '.format(model_reg.score(X, y))
    
    # accuracy score = mean(validation_score) +/- std(validation_score)
    accuracy = '{0:.3f}+/-{1:.3f}'.format(np.mean(model_val_score), np.std(model_val_score))
    return accuracy, model_r2_score


In [14]:
"""
    Evaluate the parameters
    params:
    -------------
    model: name for model, (Ridge, Lasso)
    X: descriptive features
    y: target features
    strength: default 0, strength for penalty
    
    returns: 
    --------------
    dataframe to show table of results
    
"""

def param_evaluation(model, X, y, strength):
    scores = []
    fold = [5, 10, 20]
    for k in fold: 
        for alpha in strength:
            accuracy, r2_score = model_validation(model, X, y, k, alpha)
            score = [k, alpha, accuracy, r2_score]
            scores.append(score)
    return pd.DataFrame(np.array(scores), columns=['fold', 'Regularization strength', 'accuracy', 'r2 score'])

Combine the results one dataframe to compare them.

In [15]:
strength = [1, 50, 100, 150, 200]
ridge_param = param_evaluation(Ridge, X, y, strength)

strength = [0.1, 0.2, 0.3, 0.4, 0.5]
lasso_param = param_evaluation(Lasso, X, y, strength)

In [16]:
### Combine the results one dataframe
pd.concat([ridge_param, lasso_param],keys=['Ridge','Lasso'],axis=1,join='inner')

Ridge                                                 Lasso  \
    fold Regularization strength       accuracy r2 score  fold   
0   5     1                       0.661+/-0.104  0.699    5      
1   5     50                      0.651+/-0.091  0.684    5      
2   5     100                     0.625+/-0.081  0.660    5      
3   5     150                     0.596+/-0.074  0.635    5      
4   5     200                     0.567+/-0.069  0.610    5      
5   10    1                       0.617+/-0.133  0.699    10     
6   10    50                      0.636+/-0.104  0.684    10     
7   10    100                     0.621+/-0.095  0.660    10     
8   10    150                     0.598+/-0.090  0.635    10     
9   10    200                     0.573+/-0.087  0.610    10     
10  20    1                       0.570+/-0.271  0.699    20     
11  20    50                      0.591+/-0.250  0.684    20     
12  20    100                     0.579+/-0.232  0.660    20     
13  20    150                     0.559+/-0.219  0.635    20     
14  20    200                     0.537+/-0.210  0.610    20     

                                                    
   Regularization strength       accuracy r2 score  
0   0.1                     0.599+/-0.080  0.620    
1   0.2                     0.538+/-0.057  0.555    
2   0.3                     0.437+/-0.048  0.459    
3   0.4                     0.297+/-0.047  0.326    
4   0.5                     0.166+/-0.022  0.174    
5   0.1                     0.579+/-0.097  0.620    
6   0.2                     0.522+/-0.069  0.555    
7   0.3                     0.422+/-0.061  0.459    
8   0.4                     0.278+/-0.056  0.326    
9   0.5                     0.135+/-0.033  0.174    
10  0.1                     0.541+/-0.274  0.620    
11  0.2                     0.486+/-0.224  0.555    
12  0.3                     0.384+/-0.180  0.459    
13  0.4                     0.238+/-0.142  0.326    
14  0.5                     0.090+/-0.111  0.174

### Implement Forward and Backward Selection algorithms
And apply it to the (given subset of variables of the) Quality of Government dataset.
Compare the results of Forward and Backward selection with each other.


In [17]:
"""
    Add one more predictor
    params: 
    ------------
    X: descriptive features
    y: target features
    old_predictors: already existing predictors
    p: the chosen predictor to add
    
    returns: 
    -------------
    dictionay of 'predictor', 'model' and 'sse'
"""
def add_predictor(X, y, old_predictors, p):
    new_predictors = old_predictors + [p]
    model = sm.OLS(y, X[new_predictors]) # linear regression to generate model
    lin_reg = model.fit()
    sse = np.sum((lin_reg.predict(X[new_predictors]) - y) ** 2)
    return {'predictor':p, 'model':lin_reg, 'sse':sse}

In [18]:
"""
    Forward algorithm to select best model
    Ref: http://www.science.smith.edu/~jcrouser/SDS293/labs/2016/lab8/Lab%208%20-%20Subset%20Selection%20in%20Python.pdf
    
    params: 
    --------------
    X: descriptive features
    y: target features
    old_predictors: already existing predictors
    mse_full: mean square error while using all variables
    
    returns: 
    best_model: best model after applying the algorithm
    mallow_cp: mallow cp value of the best model
"""

def forward(X, y, old_predictors, mse_full):
    remain_predictors = [p for p in X.columns if p not in old_predictors]
    results = []
    for p in remain_predictors:
        reg_model = add_predictor(X, y, old_predictors, p)
        results.append(reg_model)
    
    models = pd.DataFrame(results)
    best_model = models.loc[models['sse'].idxmin()] # get the best model (sse increast least)
    best_predictor = best_model['predictor']
    # mallow cp value for the best model
    mallow_cp = best_model['sse'] / mse_full - X.shape[0] + 2 * (len(old_predictors)+1)
    return best_model, mallow_cp

In [19]:
"""
    Apply forward model selection
    
    params: 
    -------------
    X: descriptive features
    y: target features
    
    returns:
    --------------
    forward_model: a list of model choosen after adding predictors
"""

def forward_model_select(X, y):

    model = sm.OLS(y, X)
    lin_reg = model.fit()
    pred_y = lin_reg.predict(X)
    # mean squared error while using all variables
    mse_full = np.mean(np.square(pred_y - y))

    forward_model_table = {}
    old_predictors = []
    for index in range(X.shape[1]):
        best_model, mallow_cp = forward(X, y, old_predictors, mse_full)
        best_predictor = best_model['predictor']
        old_predictors.append(best_predictor)
        forward_model_table[index] = (mallow_cp, best_model['sse'], tuple(old_predictors))
    
    forward_model = pd.DataFrame(forward_model_table).transpose()
    forward_model.columns = ['Mallow Cp', 'SSE', 'Predictors']
    forward_model = forward_model.sort_values('Mallow Cp')
    return forward_model # model 9 is the best

In [20]:
df_selection = df.copy()
df_selection = df_selection.fillna(df_selection.mean())# fill na values with mean value

y = df_selection['wdi_lifexp'].values
X = df_selection.drop(['wdi_lifexp'], axis=1)
predictors = X.columns
X = preprocessing.scale(X.values)
y = preprocessing.scale(y)
X = pd.DataFrame(X, columns=predictors)

In [21]:
forward_model_select(X, y) # model 9 is the best according to forward model selection

,Mallow Cp,SSE,Predictors
9,22.8765,59.1752,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden, al_ethnic, vdem_gender, bti_acp)"
10,23.3403,58.7135,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden, al_ethnic, vdem_gender, bti_acp, bti_pdi)"
8,24.0189,60.1197,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden, al_ethnic, vdem_gender)"
11,24.3039,58.402,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden, al_ethnic, vdem_gender, bti_acp, bti_pdi, wdi_forest)"
7,25.0354,61.0264,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden, al_ethnic)"
12,26.0027,58.3114,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden, al_ethnic, vdem_gender, bti_acp, bti_pdi, wdi_forest, bti_foe)"
6,26.3675,62.0279,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden)"
13,28,58.3106,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion, wdi_popden, al_ethnic, vdem_gender, bti_acp, bti_pdi, wdi_forest, bti_foe, wdi_araland)"
5,30.3667,63.8311,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci, al_religion)"
4,35.8918,66.0929,"(fh_pair, al_language, gle_cgdpc, bti_aar, bti_ci)"


In [22]:
"""
    Remove one more predictor
    params: 
    ------------
    X: descriptive features
    y: target features
    old_predictors: already existing predictors
    p: the chosen predictor to remove
    
    returns: 
    -------------
    dictionay of 'predictor', 'model' and 'sse'

"""
def remove_predictor(X, y, old_predictors, p):
    new_predictors = [s for s in old_predictors if s != p]
    model = sm.OLS(y, X[new_predictors])
    lin_reg = model.fit()
    sse = np.sum((lin_reg.predict(X[new_predictors]) - y) ** 2)
    return {'predictor':p, 'model':lin_reg, 'sse':sse}

In [23]:
"""
    Backward algorithm to select best model
    params: 
    --------------
    X: descriptive features
    y: target features
    old_predictors: already existing predictors
    mse_full: mean square error while using all variables
    
    returns: 
    best_model: best model after applying the algorithm
    mallow_cp: mallow cp value of the best model
"""

def backward(X, y, old_predictors, mse_full):
#     remain_predictors = old_predictors
    results = []
    for p in old_predictors:
        reg_model = remove_predictor(X, y, old_predictors, p)
        results.append(reg_model)
    
    models = pd.DataFrame(results)
    best_model = models.loc[models['sse'].idxmin()]
    best_predictor = best_model['predictor']
    mallow_cp = best_model['sse'] / mse_full - X.shape[0] + 2 * (len(old_predictors)+1)
    return best_model, mallow_cp

In [24]:
"""
    Apply backward model selection
    
    params: 
    -------------
    X: descriptive features
    y: target features
    
    returns:
    --------------
    backward_model: a list of model choosen after removing predictors
"""

def backward_model_select(X, y):
    backward_model_table = {}
    old_predictors = X.columns
    
    model = sm.OLS(y, X)
    lin_reg = model.fit()
    pred_y = lin_reg.predict(X)
    mse_full = np.mean(np.square(pred_y - y))

    for index in range(X.shape[1]-1):
        best_model, mallow_cp = backward(X, y, old_predictors, mse_full)
        best_predictor = best_model['predictor']
        
        ### should remove p from old predictor first, then add it to the output table.
        old_predictors = [p for p in old_predictors if p != best_predictor]
        backward_model_table[index] = (mallow_cp, best_model['sse'], tuple(old_predictors))
    
    backward_model = pd.DataFrame(backward_model_table).transpose()
    backward_model.columns = ['Mallow Cp', 'SSE', 'Predictors']
    backward_model = backward_model.sort_values('Mallow Cp')
    return backward_model # model 3 is the best

In [25]:
backward_model_select(X, y) # model 3 is the best according to forward model selection

,Mallow Cp,SSE,Predictors
3,26.8765,59.1752,"(wdi_popden, gle_cgdpc, bti_acp, fh_pair, al_ethnic, al_language, al_religion, bti_aar, vdem_gender, bti_ci)"
2,27.3403,58.7135,"(wdi_popden, gle_cgdpc, bti_acp, bti_pdi, fh_pair, al_ethnic, al_language, al_religion, bti_aar, vdem_gender, bti_ci)"
4,27.8603,60.0721,"(wdi_popden, gle_cgdpc, bti_acp, fh_pair, al_ethnic, al_language, al_religion, bti_aar, vdem_gender)"
1,28.3039,58.402,"(wdi_popden, gle_cgdpc, bti_acp, bti_pdi, fh_pair, al_ethnic, al_language, al_religion, bti_aar, vdem_gender, bti_ci, wdi_forest)"
0,30.0027,58.3114,"(wdi_popden, gle_cgdpc, bti_acp, bti_pdi, fh_pair, al_ethnic, al_language, al_religion, bti_aar, vdem_gender, bti_ci, bti_foe, wdi_forest)"
5,30.1679,61.3668,"(wdi_popden, gle_cgdpc, bti_acp, fh_pair, al_ethnic, al_language, al_religion, bti_aar)"
6,32.5081,62.6713,"(wdi_popden, gle_cgdpc, bti_acp, fh_pair, al_language, al_religion, bti_aar)"
7,37.6671,64.8231,"(wdi_popden, gle_cgdpc, bti_acp, fh_pair, al_language, bti_aar)"
8,42.887,66.9932,"(gle_cgdpc, bti_acp, fh_pair, al_language, bti_aar)"
9,49.5769,69.6051,"(gle_cgdpc, fh_pair, al_language, bti_aar)"


### Compare the result
|Selection method||||||||||||
|------ | ------ | ------ |------ |------ |------ |------ |------ |------ |------ |------ |------ |
| forward selection |al_language|al_religion|al_ethnic|bti_aar|bti_ci|bti_acp|fh_pair|gle_cgdpc|wdi_popden|vdem_gender|
|backward selection|al_language|al_religion|al_ethnic|bti_aar|bti_ci|bti_acp|fh_pair|gle_cgdpc|wdi_popden|vdem_gender|

Same result.